In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd 

from torch.utils.data import Dataset, TensorDataset, DataLoader

from transformers import BertTokenizer, BertForSequenceClassification, BertTokenizerFast

# Try with ISO-8859-1
try:
    data = pd.read_csv('/Users/askarembulatov/Github_projects/NLP_task/NLP/Notebooks/classification_problem/Corona_NLP_train.csv', encoding='ISO-8859-1')
    print("File read successfully with ISO-8859-1 encoding.")
except UnicodeDecodeError as e:
    print(f"Failed to read file with ISO-8859-1: {e}")

# If the above fails, try with Windows-1252
try:
    data = pd.read_csv('/Users/askarembulatov/Github_projects/NLP_task/NLP/Notebooks/classification_problem/Corona_NLP_train.csv', encoding='cp1252')
    print("File read successfully with Windows-1252 encoding.")
except UnicodeDecodeError as e:
    print(f"Failed to read file with Windows-1252: {e}")

data['Target'] = data['Sentiment'].replace({value:key for key, value in enumerate(data['Sentiment'].unique())})

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/Users/askarembulatov/Github_projects/NLP_task/NLP/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


File read successfully with ISO-8859-1 encoding.
Failed to read file with Windows-1252: 'charmap' codec can't decode byte 0x81 in position 153284: character maps to <undefined>


/var/folders/ws/sl0l06mn5kv5v699q9j1sgy80000gn/T/ipykernel_11164/1652698398.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Target'] = data['Sentiment'].replace({value:key for key, value in enumerate(data['Sentiment'].unique())})


In [2]:
def tokenize(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=520,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    return encoding

# Define dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        values = self.texts[idx]
        label = self.labels[idx]

        embedded = tokenize(values)

        input_ids = embedded['input_ids'].flatten()
        masks = embedded['attention_mask'].flatten()

        return {'input_ids': input_ids, 'masks': masks, 'labels': torch.tensor(label, dtype=torch.long)}


class SimpleLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(SimpleLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        
        # Activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x)

        # Pass through the fully connected layer
        out = self.fc(hn)
        
        # Apply the ReLU activation function
        out = self.relu(out)
        
        return out


# Parameters for the model
input_size = 520  # The number of features in the input (e.g., size of the embedding vector)
hidden_size = 100  # The number of features in the hidden state of the LSTM
output_size = 5  # The size of the output, e.g., 1 for a regression task

model = SimpleLSTMModel(input_size, hidden_size, output_size, 10).to(device)
dataset = TextClassificationDataset(data['OriginalTweet'].values, data['Target'].values, tokenizer)

train_dataloader = DataLoader(dataset, batch_size=10, shuffle=True, drop_last=True)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

gradient_norms = {}

# Функция для инициализации ключей в словаре
def init_gradient_norms(model):
    for name, parameter in model.named_parameters():
        if parameter.requires_grad:
            gradient_norms[name] = []

# Функция для сохранения норм градиентов после каждого обратного распространения
def save_gradients(model):
    for name, parameter in model.named_parameters():
        if parameter.grad is not None:
            gradient_norms[name].append(parameter.grad.norm().item())

init_gradient_norms(model)

for epoch in range(10):
    # Forward pass
    total = 0
    correct = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(torch.float32)
        attention_mask = batch['masks']
        labels = batch['labels']

        input_ids, masks, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids).squeeze()
        loss = criterion(outputs, labels)
        
        # Backward pass и оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        save_gradients(model)  # Печать нормы градиентов
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(accuracy)

0.2305224787363305
0.24267314702308626
